# Phenotyping algorithm clustering 

This notebook is to cluster the sentence fragments extracted from phenotyping algorithms with KMeans and Non-negative matrix factorization algorithm

In [2]:
import csv,os
import random
import string
import logging
import sys
import re
import glob
import nltk
import pickle
from numpy import linalg as LA
from sklearn.decomposition import NMF
from sklearn.cluster import KMeans
from nltk.stem.porter import *
from sklearn.metrics.cluster import adjusted_mutual_info_score
from nltk.corpus import stopwords
from collections import Counter
import numpy as np
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
from sklearn.metrics import roc_curve, auc,f1_score,adjusted_rand_score
from sklearn.cross_validation import StratifiedKFold
from pprint import pprint
from sklearn.grid_search import GridSearchCV
from optparse import OptionParser
from sklearn.cross_validation import StratifiedShuffleSplit
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer,TfidfVectorizer,HashingVectorizer
from sklearn.svm import LinearSVC,SVC
from sklearn.utils.extmath import density
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score, adjusted_mutual_info_score, roc_auc_score, mutual_info_score, make_scorer, roc_curve,accuracy_score


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [63]:
def read_csv(filepath,i):
    name = []
    with open(filepath, "rb") as csvfile:
        reader = csv.reader(csvfile)
        next(reader, None)
        for row in reader:
            if len(row) == 0:
                continue
            name.append(row[i])
    return name

#find the row which is 1
def attach(feature,list):
    index = [i for i, j in enumerate(list) if j == '1']
    text = [feature[i] for i in index] 
    return text

def match_feature(whole_text,traintext,testtext,feature):
	train = []
	test = []
	#print traintext
	for i in traintext:
		train.append(feature[whole_text.index(i)])
	for i in testtext:
		test.append(feature[whole_text.index(i)])
	return train,test

def makeSiteMatrix(wholesite,site_category):
    site_matrix = []
    for site in wholesite:
        row = []
        for c in site_category:
            if c == site:
                row.append(1)
            else:
                row.append(0)
        site_matrix.append(row)
    return site_matrix

def removeNonAscii(s): 
    """
    s should be a utf-8 encoded string
    """
    return "".join(i for i in s if ord(i)<128)

def token(i):
    a =  nltk.word_tokenize(i)
    words = []
    for j in a:
        words.append(stemmer.stem(j))
    return words 

def token_unstem(i):
    a =  nltk.word_tokenize(i)
    return a


def words( TEXT ):
    train = [] 
    #print TEXT  
    for si in TEXT:
        i = removeNonAscii(si)
        i = i.lower()
        #words = re.sub("[/*'-]", " ",i)
        words = re.sub("[/'-]", " ",i) #replace with space
        words = words.replace("[", " ")
        words = words.replace("]", " ")
        words = words.replace("(", " ")
        words = words.replace(")", " ")   
        words = words.replace("=", " ")   
        words = words.replace(">", " ") 
        words = words.replace("*", " ")
        words = words.replace("+", " ")
        words = re.sub(r"(^|\s)[0-9]+[.][0-9x*_.]+($|\s)", " _number_ ", words) #replace icd9 codes with icd_code   
        words = re.sub(r"(^|\s)[,.\-_>=<+?*/&\"\'()\[\]]*[0-9]+[,.\-_>=<+?*/&\"\'()\[\]]*($|\s)", " _number_ ", words) #replace single number to _number_
        words = re.sub(r"(^|\s)[0-9]+($|\s)", " _number_ ", words)    #replace single number with number
        words = re.sub(r"\b(one|two|three|four)\b","_number_",words) #replace word number to _number_
        train.append(words)
    return train

def binary(values,l,name):
    for ss in range(len(values)):
        if name in values[ss]:
            l.append(1)
        else:
            l.append(0)
    return l

def addmatrix(tfs_train_array,matrix,features,addfeatures):
    matrix = preprocessing.normalize(matrix, norm='l2')  
    tfs_train = np.hstack((tfs_train_array,matrix))   
    features = features + addfeatures 
    return tfs_train,features

def bag_of_words(text):
    tfs = countvectorizer.fit_transform(text)    
    features_words = countvectorizer.get_feature_names()
    tfs = preprocessing.normalize(tfs, norm='l2') 
    tfs = tfs.toarray()
    return tfs,features_words

def parse(file):
    codelist = []
    allcode = []
    textlist = []
    match = {}
    temp = []
    temp_stype=[]
    temp_exp = {}
    stype = []
    stype_list = []
    with open(file) as f:
        for line in f:
            if line[0:24] == "Processing 00000000.tx.1":
                temp = list(set(temp))
                if temp != []:
                    codelist = codelist + temp                   
                temp = []
                temp_stype = list(set(temp_stype))
                # print temp_stype
                if temp_stype != []:
                    stype_list = stype_list + temp_stype
                temp_stype = []

                if codelist != []:
                    allcode.append(codelist)
                    #all_exp.append(exp_list)
                if stype_list != []:
                    stype.append(stype_list)
                # print stype_list
                text = line[26:]
                textlist.append(text.rstrip())
                codelist = []
                stype_list = []
                # print "-----------------------------"
            elif line[0:6] == "Phrase":
                temp = list(set(temp))
                #temp_exp = list(set(temp_exp))
                if temp != []:
                    codelist = codelist + temp
                    #z = temp_exp.copy()
                    #z.update(codelist)
                temp = []

                temp_stype = list(set(temp_stype))
                # print temp_stype
                if temp_stype != []:
                    stype_list = stype_list + temp_stype
                temp_stype = []
            else:
                my = re.compile("C[0-9]{7}")
                # print line
                result = my.findall(line)
                if result != []:
                    temp.append(result[0])
                    a = line.index(":")
                    temp_exp[result[0]] = str(line[a+1:-1])

                my2 = re.compile("\[.+\]")
                result = my2.findall(line)
                # print line
                if result != []:
                    temp_stype.append(result[0][1:-1])
                    # print result[0]

        codelist = codelist + list(set(temp))
        # print temp_stype
        stype_list = stype_list + list(set(temp_stype))
        allcode.append(codelist)
        stype.append(stype_list)
    return allcode, textlist, temp_exp, stype

#union of all codes
def getunion(umls):
    l = []
    for i in umls:
        l = list(set(l+i))
    return l

def umlsfeature(allcode,allwords):
    whole = []
    for i in allcode:
        freq = []
        for j in allwords:
            if j in i:
                freq.append(i.count(j))
            else:
                freq.append(0)
        whole.append(freq)
    whole = np.array(whole)
    return whole

def findOccurance(text, string):
    occ = []
    for i in text:
        a = len([m.start() for m in re.finditer(string, i)])
        occ.append(a)
    return occ

def reshape(x):
    x1, x_cate = np.unique(x,return_inverse = True)
    x_cate = x_cate.reshape(len(x),1)
    return x_cate


def embedding(tfs,numberOcc_train,mem):
    tfs_train = np.dot(tfs, mem)#get train matrix 
    numberOcc_train = reshape(numberOcc_train)
    tfs_train = np.hstack((tfs_train,numberOcc_train)) 
    tfs_train = preprocessing.normalize(tfs_train, norm='l2')
    #get combined feature
    features = list(range(200)) + ["number"]  
    return tfs_train,features



def subset(worddict,features,mem):
    subset_embedding = []
    for ss in features:
        if ss in worddict:
            subset_embedding.append(mem[worddict[ss]])
        else:
            print("word not embedded",ss)
            subset_embedding.append([0] * len(mem[1]))
    subset_embedding = np.asarray(subset_embedding)
    return subset_embedding

def count_f1(count_site):
    #TP,FP,TN,FN
    TP = 0
    FP = 0
    FN = 0
    pre = []
    recal = []
    for lis in count_site:
        TP = TP + lis[0]
        FP = FP + lis[1]
        FN = FN + lis[3]
        if lis[0] + lis[1] == 0:
            pre.append(0)
        else:
            pre.append(lis[0]/float((lis[0]+lis[1])))
        recal.append(lis[0]/float((lis[0]+lis[3])))
  
    if TP + FP == 0:
        micro_pres = 0
    else:
        micro_pres = float(TP)/(TP + FP)
    #print micro_pres
    micro_recall = float(TP)/(TP + FN)
    #print micro_recall
    f1 = 2*micro_pres*micro_recall/(micro_pres + micro_recall)
    macro_pre = sum(pre)/len(count_site) 
    macro_recall = sum(recal)/len(count_site)
    macro_f1 = 2*macro_pre*macro_recall/(macro_pre + macro_recall)
    return macro_f1,f1




def makeBinaryMatrix(train_umls):
    binary_matrix = []
    for i in range(len(train_umls)):
        row = []
        for j in range(len(train_umls[i])):
            if train_umls[i][j] != 0:
                row.append(1)
            else:
                row.append(0)
        binary_matrix.append(row)
    return binary_matrix
    
def readtensor(filename):
    F = open(filename,"r")
    rank = []
    for line in F:
        line = line.rstrip()
        ss = line.split(" ")
        ss = [float(i) for i in ss]
        rank.append(ss)
        #rank_10 = np.asarray(rank_10)
    return rank

def printfeature(feature_matrix, feature):
    l = feature_matrix.tolist()
    index = [i for i, j in enumerate(l) if j != 0]
    for j in index:
        print(l[j], feature[j])
    #text = [feature[i] for i in index] 
    #print text
    return text

def perf_measure(y_actual, y_hat):
    COUNT = []
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
           TP += 1

    for i in range(len(y_hat)): 
        if y_actual[i]==0 and y_hat[i] == 1:
           FP += 1
    
    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==0:
           TN += 1
    
    for i in range(len(y_hat)): 
        if y_actual[i]==1 and y_hat[i] == 0:
           FN += 1
    return [TP,FP,TN,FN]

def writecnninput(label,pos_out,neg_out,pos,neg):
    for i in pos[label]:
        pos_out.writelines(i+os.linesep)
    for i in neg[label]:
        neg_out.writelines(i+os.linesep)

def getsubset(name,text,pattern,site,algo):
    #######get subset index
    index = []
    text_sub = []
    site_sub = []
    pattern_sub = []
    algo_sub = []
    for i in range(len(pattern)):
        if pattern[i] in name:
            text_sub.append(text[i])
            pattern_sub.append(pattern[i])
            site_sub.append(site[i])
            algo_sub.append(algo[i])
        else:
            #print(pattern[i])
            continue
    return text_sub,pattern_sub, site_sub, algo_sub


def cluster(estimator, data, pred_label):
    estimator.fit(data)
    #print estimator.labels_
    #print("Adjusted Rand-Index: %.3f"
    #  % adjusted_rand_score(pred_label, estimator.labels_))
    return estimator.labels_


### Data

Read data and collect features (algorithm developing sites and algorithm)

In [17]:
text = []
pattern = []
site = []
algo = []
with open("../data/sentence_all_info_0817.csv") as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            if len(row) == 0:
                continue
            text.append(row[0].strip())
            pattern.append(row[1])
            site.append(row[2])
            algo.append(row[3])
            
print("number of all sentence:", len(text))

number of all sentence: 190


Convert the sentences to a dict, in which the key is the sentence and value is the label

In [18]:
match = {}
for i in range(len(text)):
    if text[i] in match:
        match[text[i]].append(pattern[i])
      
    else:
        match[text[i]] = [pattern[i]]

print("number of unique sentence:", len(match.keys()))

number of unique sentence: 154


Only use 6 classes with large number of sentence 

In [19]:
#name = ["Medication Details","Confirm Disease Was Checked","Rule of N","Use Distinct Dates","Level of Evidence","Credentials of the Actor","Where Did It Happen?","Check For Negation"]#
name = ["Confirm Disease Was Checked","Rule of N","Use Distinct Dates","Credentials of the Actor","Where Did It Happen?","Check For Negation"]#
#name = ["Rule of N","Use Distinct Dates"]
text_sub,pattern_sub,site_sub,algo_sub = getsubset(name,text,pattern,site,algo)
unique_text_sub = list(set(text_sub))


match_sub = {}
for i in range(len(text_sub)):
    if text_sub[i] in match_sub:
        match_sub[text_sub[i]].append(pattern_sub[i])
      
    else:
        match_sub[text_sub[i]] = [pattern_sub[i]]

    
unique_pattern = []
for i in match_sub.values():
    unique_pattern.append(i[0])

print("number of unique sentence after subsetting:", len(match_sub.keys()))  



number of unique sentence after subsetting: 131


In [20]:
#get the sentence that has only one label
new_match = {}
for k, v in match_sub.items():
    if len(v) == 1:
        new_match[k] = v
        
text_unique = new_match.keys()
train_text = words(text_unique)

label_list = new_match.values()

print("number of sentences with unique label: ", len(text_unique))

number of sentences with unique label:  101


In [21]:
labels = []
for i in label_list:
    labels.append(i[0])
le = preprocessing.LabelEncoder()
le.fit(labels)
label = le.transform(labels) 
print(len(set(label)))

6


## Process features

In [22]:
#process CUI and ST features
#read umls file
allcode_train, textlist_train,all_exp,all_st_train = parse("../data/umls_train_0623.txt")
allcode_test,textlist_test,test_exp,all_st_test = parse("../data/umls_test_0624.txt")

all_text_umls = textlist_train + textlist_test
all_cui_umls = allcode_train + allcode_test
#all_exp_umls = all_exp + test_exp
all_st_umls = all_st_train + all_st_test

allcode_train = list(map(lambda x: all_cui_umls[all_text_umls.index(x)], text_unique))
all_st_train = list(map(lambda x: all_st_umls[all_text_umls.index(x)], text_unique))

#get the union of all CUI code and st
CUI = getunion(allcode_train)
st = getunion(all_st_train)

all_train_CUI = umlsfeature(allcode_train,CUI)
all_train_st = umlsfeature(all_st_train,st)




In [24]:
#process site and algo
site_train = list(map(lambda x: site[text.index(x)], text_unique))
algo_train = list(map(lambda x: algo[text.index(x)], text_unique))


#process site and algorithm features
site_category = list(set(site_train))
algo_category = list(set(algo_train))


all_train_site = makeSiteMatrix(site_train,site_category)
all_train_algo = makeSiteMatrix(algo_train,algo_category)



In [59]:
#process embedding
import sys  

#reload(sys)  
#sys.setdefaultencoding('utf8')
fn = '../data/embedding/embedding_mimic3_pp200_0829.pik'
f = open(fn, 'rb')
a = pickle.load(f, encoding='bytes')
(mem, hwoov, hwid) = a

#subset_mem = subset(embedding_word,features,mem)
subset_mem = np.zeros((len(features), 200))
(mem, hwoov, hwid) = a
for i in range(len(features)):
    try:
        subset_mem[i,:] = mem[hwid[str.encode(features[i])],:]
    except:
        continue

numberOcc_train = findOccurance(train_text,"_number_")         


## Model

In [40]:
seeds = [1,10,100,500,550,1000,1500,2000,3000,4000]

def kmeansClustering(train):
    c = []
    for seed in seeds:
        kmeans = KMeans(n_clusters=6, init='k-means++', n_init=10, max_iter=300, tol=0.0001, precompute_distances='auto', 
        verbose=0, random_state=seed, copy_x=True, n_jobs=-1)
        cluster0 = cluster(kmeans,train,label)
        c.append(cluster0)
    return c

def nmf_clustering(train):
    c = []
    if np.amin(train) < 0:
        train = train - np.amin(train)
    #print np.amin(train)
    train_nmf = np.transpose(train)
    for seed in seeds:
        model = NMF(n_components=6, init='random', random_state=seed)
        model.fit(train_nmf) 
        nmf = model.components_ 
        nmf = np.transpose(nmf)
        ll = [] 
        for kk in range(nmf.shape[0]):
            l = nmf[kk,]
            ss = l.tolist()
            ll.append(ss.index(max(ss)))
        c.append(ll)
    #print c
    return c

def ami_eva(gt, predict):
    score = []
    for li in predict:
        score.append(adjusted_mutual_info_score(gt, li))    
    return max(score)

In [27]:
#nlp training and testing
stop = []
f = open("stoplist3.txt")
stop = f.read().splitlines() #stop list containing have
stemmer = PorterStemmer()
countvectorizer = TfidfVectorizer(tokenizer=token, stop_words=stop,decode_error ="ignore")
#countvectorizer = CountVectorizer(tokenizer=token, stop_words=stop,decode_error ="ignore",binary = True)

#print text
train, features = bag_of_words(train_text)


In [42]:
print("NMF", ami_eva(label,nmf_clustering(train)))
print("Kmeans", ami_eva(label,kmeansClustering(train)))

NMF 0.406583864016
Kmeans 0.226991043844


In [47]:
x_train_site,features_site = addmatrix(train,all_train_site,features,site_category)
print("NMF", ami_eva(label,nmf_clustering(x_train_site)))
print("Kmeans", ami_eva(label,kmeansClustering(x_train_site)))

NMF 0.144278283197
Kmeans 0.133392331879


In [48]:
x_train_algo,features_algo = addmatrix(train,all_train_algo,features,algo_category)
print("NMF", ami_eva(label,nmf_clustering(x_train_algo)))
print("Kmeans", ami_eva(label,kmeansClustering(x_train_algo)))

NMF 0.180324205561
Kmeans 0.224686866289


In [50]:
x_train_CUI,features_CUI = addmatrix(train,all_train_CUI,features,CUI)
print("NMF", ami_eva(label,nmf_clustering(x_train_CUI)))
print("Kmeans", ami_eva(label,kmeansClustering(x_train_CUI)))

NMF 0.33562822012
Kmeans 0.214373448414


In [53]:
x_train_st,features_st = addmatrix(train,all_train_st,features,st)
print("NMF", ami_eva(label,nmf_clustering(x_train_st)))
print("Kmeans", ami_eva(label,kmeansClustering(x_train_st)))

NMF 0.127200036854
Kmeans 0.244136558757


In [65]:
em, feature_em = embedding(train,numberOcc_train,subset_mem)
print("NMF", ami_eva(label,nmf_clustering(em)))
print("Kmeans", ami_eva(label,kmeansClustering(em)))

NMF 0.124545250932
Kmeans 0.167508760165


In [66]:
x_em_site,features_site_em = addmatrix(em,all_train_site,feature_em,site_category)
print("NMF", ami_eva(label,nmf_clustering(x_em_site)))
print("Kmeans", ami_eva(label,kmeansClustering(x_em_site)))

NMF 0.126855255587
Kmeans 0.133392331879


In [67]:
x_em_algo,features_algo_em = addmatrix(em,all_train_algo,feature_em,algo_category)
print("NMF", ami_eva(label,nmf_clustering(x_em_algo)))
print("Kmeans", ami_eva(label,kmeansClustering(x_em_algo)))

NMF 0.188213482531
Kmeans 0.131965253632


In [68]:
x_em_CUI,features_CUI_em = addmatrix(em,all_train_CUI,feature_em,CUI)
print("NMF", ami_eva(label,nmf_clustering(x_em_CUI)))
print("Kmeans", ami_eva(label,kmeansClustering(x_em_CUI)))

NMF 0.203821443197
Kmeans 0.199827495258


In [69]:
x_em_st,features_st_em = addmatrix(em,all_train_st,feature_em,st)
print("NMF", ami_eva(label,nmf_clustering(x_em_st)))
print("Kmeans", ami_eva(label,kmeansClustering(x_em_st)))

NMF 0.135492435157
Kmeans 0.187575890893
